In [4]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfgeXvsuZX7EF_FIqgFoldJyAkpLPINQ1Nt-bQ1UBbWotfq2hFXNb0BbuIU

Successfully saved authorization token.


In [17]:
def clipper(image, shp_file):
    return image.clip(shp_file)

def spei(dataset):
    precipitation = dataset.select('pr').rename('Pp')
    pet = dataset.select('pet').multiply(0.1).rename('PET')
    
    time = dataset.get('system:time_start')
    
    return precipitation.subtract(pet).multiply(0.1).rename('SPEI').set('system:time_start', time)

In [12]:
shp_argentina = ee.FeatureCollection('users/adelarja/Hidro2022/Prov_Arg')

In [31]:
start_date = ee.Date.fromYMD(2000, 1, 1)
end_date = ee.Date.fromYMD(2020, 1, 1)

In [32]:
dataset = ee.ImageCollection(
    'IDAHO_EPSCOR/TERRACLIMATE').filterDate(
    start_date, end_date).map(
    lambda x: clipper(x, shp_argentina)
)

dataset_spei = dataset.map(spei)

In [56]:
 palette = [
    '1a3678', '2955bc', '5699ff', '8dbae9', 'acd1ff', 'caebff', 'e5f9ff',
    'fdffb4', 'ffe6a2', 'ffc969', 'ffa12d', 'ff7c1f', 'ca531a', 'ff0000',
    'ab0000'
]
petVis = {
    'min': -2,
    'max': 2,
    'palette': palette
}

In [57]:
import folium

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [58]:
lat, lon = -34.603722, -58.381592

my_map = folium.Map(location=[lat, lon], zoom_start=5)

my_map.add_ee_layer(dataset_spei.mean(), petVis, "SPEI")
my_map.add_ee_layer(shp_argentina, {}, "Argentina")
my_map.add_child(folium.LayerControl())
display(my_map)